# Assortment_Optimization_System

### Import Libraries

In [23]:
import pandas as pd
from src.utils import data_validate

## STEP-1: Load All Dataset

In [13]:

sales = pd.read_csv("../Data/sample_sales_multi_store.csv", parse_dates=["date"])
products = pd.read_csv("../Data/sample_products.csv")
stores = pd.read_csv("../Data/sample_stores.csv")

sales.head()

,store_id,date,product_id,product_name,units_sold,revenue,cost,price
0,S001,2025-01-05,P001,Coca-Cola 500ml,33,1689.58,1091.23,51.20
1,S001,2025-01-05,P002,Pepsi 500ml,25,2394.15,1529.87,95.77
2,S001,2025-01-05,P003,Lays Chips 100g,15,407.39,291.21,27.16
3,S001,2025-01-05,P004,Doritos Nacho 100g,44,2055.15,1596.89,46.71
4,S001,2025-01-05,P005,Oreo Biscuit 120g,6,73.59,57.43,12.26


In [14]:
products.head()

,product_id,product_name,category,price
0,P001,Coca-Cola 500ml,Beverages,35
1,P002,Pepsi 500ml,Beverages,34
2,P003,Lays Chips 100g,Snacks,20
3,P004,Doritos Nacho 100g,Snacks,25
4,P005,Oreo Biscuit 120g,Biscuits,30


In [15]:
stores.head()

,store_id,store_name,city,store_type
0,S001,Mumbai Store,Metro City,HighStreet
1,S002,Bangalore Store,Urban City,Supermarket
2,S003,Pune Store,Urban City,Convenience


In [16]:
print("Shapes(sales, products, stores):", sales.shape, products.shape, stores.shape)

Shapes(sales, products, stores): (240, 8) (10, 4) (3, 4)


## STEP-2: Data Quality Check

In [17]:
print("Sales columns:", sales.columns.tolist())
print("Products columns:", products.columns.tolist())
print("Store columns:", stores.columns.tolist())

print("\nMissing values in sales:\n", sales.isnull().sum())
print("\nMissing values in products:\n", products.isnull().sum())
print("\nMissing values in stores:\n", stores.isnull().sum())

print("\nData Types:\n", sales.dtypes)

Sales columns: ['store_id', 'date', 'product_id', 'product_name', 'units_sold', 'revenue', 'cost', 'price']
Products columns: ['product_id', 'product_name', 'category', 'price']
Store columns: ['store_id', 'store_name', 'city', 'store_type']

Missing values in sales:
 store_id        0
date            0
product_id      0
product_name    0
units_sold      0
revenue         0
cost            0
price           0
dtype: int64

Missing values in products:
 product_id      0
product_name    0
category        0
price           0
dtype: int64

Missing values in stores:
 store_id      0
store_name    0
city          0
store_type    0
dtype: int64

Data Types:
 store_id                object
date            datetime64[ns]
product_id              object
product_name            object
units_sold               int64
revenue                float64
cost                   float64
price                  float64
dtype: object


## STEP-3: Merge Sales + Product + Store Data

In [18]:
# Merge Sales with Products on product_id
sales_products = pd.merge(sales,products, on="product_id", how="left")

# Merge Result with Stores on store_id
final_df = pd.merge(sales_products, stores, on="store_id", how="left")

print("Final Merged Shape:", final_df.shape)
final_df.head()


Final Merged Shape: (240, 14)


,store_id,date,product_id,product_name_x,units_sold,revenue,cost,price_x,product_name_y,category,price_y,store_name,city,store_type
0,S001,2025-01-05,P001,Coca-Cola 500ml,33,1689.58,1091.23,51.20,Coca-Cola 500ml,Beverages,35,Mumbai Store,Metro City,HighStreet
1,S001,2025-01-05,P002,Pepsi 500ml,25,2394.15,1529.87,95.77,Pepsi 500ml,Beverages,34,Mumbai Store,Metro City,HighStreet
2,S001,2025-01-05,P003,Lays Chips 100g,15,407.39,291.21,27.16,Lays Chips 100g,Snacks,20,Mumbai Store,Metro City,HighStreet
3,S001,2025-01-05,P004,Doritos Nacho 100g,44,2055.15,1596.89,46.71,Doritos Nacho 100g,Snacks,25,Mumbai Store,Metro City,HighStreet
4,S001,2025-01-05,P005,Oreo Biscuit 120g,6,73.59,57.43,12.26,Oreo Biscuit 120g,Biscuits,30,Mumbai Store,Metro City,HighStreet


## STEP-4: Add Derived/Feature Engineering Columns

In [19]:
# Feature Engineering
final_df['week'] = final_df['date'].dt.isocalendar().week
final_df['month'] = final_df['date'].dt.month

# Calculate Profit & Profit Margin
final_df['profit'] = final_df['revenue'] - final_df['cost']
final_df['profit_margin'] = (final_df["profit"] / final_df["revenue"]).round(3)

# Revenue per Unit
final_df['revenue_per_unit'] = (final_df['revenue'] / final_df['units_sold']).round(2)

print("Added new columns: week, month, profit, profit_margin, revenue_per_unit")
final_df.head()


Added new columns: week, month, profit, profit_margin, revenue_per_unit


,store_id,date,product_id,product_name_x,units_sold,revenue,cost,price_x,product_name_y,category,price_y,store_name,city,store_type,week,month,profit,profit_margin,revenue_per_unit
0,S001,2025-01-05,P001,Coca-Cola 500ml,33,1689.58,1091.23,51.20,Coca-Cola 500ml,Beverages,35,Mumbai Store,Metro City,HighStreet,1,1,598.35,0.354,51.20
1,S001,2025-01-05,P002,Pepsi 500ml,25,2394.15,1529.87,95.77,Pepsi 500ml,Beverages,34,Mumbai Store,Metro City,HighStreet,1,1,864.28,0.361,95.77
2,S001,2025-01-05,P003,Lays Chips 100g,15,407.39,291.21,27.16,Lays Chips 100g,Snacks,20,Mumbai Store,Metro City,HighStreet,1,1,116.18,0.285,27.16
3,S001,2025-01-05,P004,Doritos Nacho 100g,44,2055.15,1596.89,46.71,Doritos Nacho 100g,Snacks,25,Mumbai Store,Metro City,HighStreet,1,1,458.26,0.223,46.71
4,S001,2025-01-05,P005,Oreo Biscuit 120g,6,73.59,57.43,12.26,Oreo Biscuit 120g,Biscuits,30,Mumbai Store,Metro City,HighStreet,1,1,16.16,0.220,12.26


In [20]:
len(final_df.columns)

19

## STEP-5: Basic Data Validation

In [21]:
data_validate(sales, "Sales Data")
data_validate(products, "Products Data")
data_validate(stores, "Store Data")


---- Sales Data Validation ----
Shape: (240, 8)
No missing values
Duplicates: 0
Columns: ['store_id', 'date', 'product_id', 'product_name', 'units_sold', 'revenue', 'cost', 'price']
----------------------------

---- Products Data Validation ----
Shape: (10, 4)
No missing values
Duplicates: 0
Columns: ['product_id', 'product_name', 'category', 'price']
----------------------------

---- Store Data Validation ----
Shape: (3, 4)
No missing values
Duplicates: 0
Columns: ['store_id', 'store_name', 'city', 'store_type']
----------------------------


In [22]:
final_df.to_csv("../Data/final_prepared.csv", index=False)
print("Saved final prepared dataset to Data/final_prepared.csv")

Saved final prepared dataset to Data/final_prepared.csv
